# Metrics experiments
- After fixing the barplot in the Jupyter notebook I decided to switch most of my metric development to a notebook

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Software versions
Python 2.7.12 64bit [GCC 5.4.0 20160609]
IPython 5.4.1
OS Linux 4.10.0 35 generic x86_64 with Ubuntu 16.04 xenial
numpy 1.13.1
scipy 0.19.1
matplotlib 2.0.2
pandas 0.20.3
Sat Sep 23 16:29:04 2017 AEST

In [2]:
import os
import sys

In [3]:
sys.path.append("/home/jack/Dev/BeeUnique/src")

In [4]:
from Processor.PathAnalysis.pathmetrics import PathMetrics
from Processor.Utils.graphics import plot_path_bg, plot_histogram, plot_line, plot_seconds_of_activity
from Processor.Utils.fileutils import create_dir_check_exists
from Processor.Utils import constants

In [6]:
%%time

experiment_directory = '/home/jack/Data/Caffeine_Unique_Tags_2'

plots_dir = create_dir_check_exists(experiment_directory, 'plots')
path_bg_dir = create_dir_check_exists(plots_dir, 'path_bg')
distances_per_second_window_dir = create_dir_check_exists(plots_dir, 'distances_per_second_window')
seconds_spent_in_perimeter_dir = create_dir_check_exists(plots_dir, 'seconds_spent_in_perimeter')
consecutive_seconds_motionless_dir = create_dir_check_exists(plots_dir, 'consecutive_seconds_motionless')
consecutive_seconds_motionless_path_dir = create_dir_check_exists(plots_dir, 'consecutive_seconds_motionless_path')

heatmaps_dir = create_dir_check_exists(plots_dir, 'heatmaps')
pm = PathMetrics(experiment_directory)

('Processed', datetime.datetime(2017, 4, 7, 1, 10, 36))
('Processed', datetime.datetime(2017, 4, 7, 2, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 3, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 4, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 5, 10, 36))
('Processed', datetime.datetime(2017, 4, 7, 6, 10, 36))
('Processed', datetime.datetime(2017, 4, 7, 7, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 8, 10, 36))
('Processed', datetime.datetime(2017, 4, 7, 9, 10, 36))
('Processed', datetime.datetime(2017, 4, 7, 10, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 11, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 12, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 13, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 14, 10, 37))
('Processed', datetime.datetime(2017, 4, 7, 15, 10, 36))
('Processed', datetime.datetime(2017, 4, 7, 16, 10, 36))
('Processed', datetime.datetime(2017, 4, 7, 17, 10, 36))
('Processed', datetime.datetime(2017, 4,

('Processed', datetime.datetime(2017, 4, 13, 1, 10, 36))
('Processed', datetime.datetime(2017, 4, 13, 2, 10, 36))
('Processed', datetime.datetime(2017, 4, 13, 3, 10, 36))
('Processed', datetime.datetime(2017, 4, 13, 4, 10, 36))
('Processed', datetime.datetime(2017, 4, 13, 5, 10, 36))
('Processed', datetime.datetime(2017, 4, 13, 6, 10, 37))
('Processed', datetime.datetime(2017, 4, 13, 7, 10, 37))
('Processed', datetime.datetime(2017, 4, 13, 8, 10, 37))
('Processed', datetime.datetime(2017, 4, 13, 9, 10, 37))
('Processed', datetime.datetime(2017, 4, 13, 10, 10, 37))
('Processed', datetime.datetime(2017, 4, 13, 11, 10, 37))
('Processed', datetime.datetime(2017, 4, 13, 12, 10, 37))
('Processed', datetime.datetime(2017, 4, 13, 13, 10, 37))
('Processed', datetime.datetime(2017, 4, 13, 14, 10, 39))
('Processed', datetime.datetime(2017, 4, 13, 14, 51, 46))
('Processed', datetime.datetime(2017, 4, 13, 16, 0, 46))
('Processed', datetime.datetime(2017, 4, 13, 17, 0, 46))
('Processed', datetime.da

('Processed', datetime.datetime(2017, 4, 19, 2, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 3, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 4, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 5, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 6, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 7, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 8, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 9, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 10, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 11, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 12, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 13, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 14, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 15, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 16, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 17, 0, 46))
('Processed', datetime.datetime(2017, 4, 19, 18, 0, 46))
('Processed', datetime.datetime(2017, 4

In [6]:
%%time

pm.group_bee_data_into_nights_days()

0
11
13
15
17
19
20
22
24
26
28
2
31
33
35
37
39
40
42
44
46
48
4
51
53
55
57
59
6
8
10
12
14
16
18
1
21
23
25
27
29
30
32
34
36
38
41
43
45
47
49
50
52
54
56
58
5
7
9
CPU times: user 1h 27min 34s, sys: 53.2 s, total: 1h 28min 27s
Wall time: 1h 28min 21s


In [8]:
pm.generate_night_day_bgs()

In [9]:
import shelve
db = shelve.open('/home/jack/Desktop/circadian_metrics.db')
bee = db[str(constants.QUEEN_CLASS)]
db.close()

In [14]:
seconds_motionless = []
dividing_lines = []

#bee = pm.tag_class_night_day_metrics[constants.QUEEN_CLASS]

for i in range(len(bee['night'])):
    night_metrics = bee['night'][i]
    day_metrics = bee['day'][i]

    for motionless_data in night_metrics.all_motionless_data:
        seconds_motionless.append(motionless_data['seconds_motionless'])

    dividing_lines.append(sum([abs(s) for s in seconds_motionless]))

    for motionless_data in day_metrics.all_motionless_data:
        seconds_motionless.append(motionless_data['seconds_motionless'])

time_period_str = "all_night_day"
file_name = os.path.join(consecutive_seconds_motionless_path_dir, time_period_str + '.png')
plot_seconds_of_activity(seconds_motionless, dividing_lines, file_name)

In [ ]:
bee = pm.tag_class_night_day_metrics[constants.QUEEN_CLASS]
    for night_day in bee.keys():
        print(night_day)
        night_or_day_count = 0
        night_or_day_id = {'night': '_0_', 'day': '_1_'}
        for i in range(len(bee[night_day])):
            print(i)
            metrics = bee[night_day][i]
            bg_image = pm.night_day_bg_images[night_day][i]

            time_period_str = str(night_or_day_count) + night_or_day_id[night_day] + night_day

            '''
            file_name = os.path.join(path_bg_dir, time_period_str + '.png')
            plot_path_bg(metrics.x_paths, metrics.y_paths, bg_image, file_name)

            file_name = os.path.join(distances_per_second_window_dir, time_period_str + '.png')
            plot_histogram(metrics.all_distances_per_second_window, 'Distance per second', file_name)

            #file_name = os.path.join(seconds_spent_in_perimeter_dir, time_period_str + '.png')
            #plot_histogram(metrics['seconds_spent_in_perimeter'], 'Seconds spent in perimeter', file_name)
            '''

            seconds_motionless = []
            for motionless_data in metrics.all_motionless_data:
                seconds_motionless.append(motionless_data['seconds_motionless'])

            file_name = os.path.join(consecutive_seconds_motionless_dir, time_period_str + '.png')
            plot_histogram(seconds_motionless, 'Consecutive seconds motionless', file_name)

            file_name = os.path.join(consecutive_seconds_motionless_path_dir, time_period_str + '.png')
            #plot_line(range(len(seconds_motionless)), seconds_motionless, 'Consecutive seconds motionless', 'Seconds in paths', 'Seconds motionless', 0, constants.SECONDS_IN_45_MINS, file_name)

            plot_barplot(seconds_motionless, range(len(seconds_motionless)), file_name, 'Consecutive seconds motionless', 'Seconds in time period', 'Seconds motionless', 0, constants.SECONDS_IN_45_MINS)

            '''
            for speed_group in metrics.cells_visited_speed_groups.keys():
                speed_group_heatmap_dir = create_dir_check_exists(heatmaps_dir, speed_group)
                file_name = os.path.join(speed_group_heatmap_dir, time_period_str)
                all_coords_spread, class_presence_spread = pm.calc_heatmaps(metrics.cells_visited_speed_groups[speed_group], bg_image, file_name)
            '''

            night_or_day_count += 1

In [ ]:
def plot_barplot(list_values, list_values_names, file_name, title, xtitle, ytitle, ymin, ymax, dividing_lines=None):
    plt.figure()

    seconds = []
    activity = []
    for s in list_values:
        if s < 0:
            activity.append('royalblue')
            seconds.append(abs(s))
            #seconds.extend([1 for i in range(abs(s))])
            #activity.extend(['absent' for i in range(abs(s))])
        elif s == 0 or s == 1:
            activity.append('k')
            seconds.append(1)
            #activity.append('moving')
        else:
            activity.append('indianred')
            seconds.append(s)
            #seconds.extend([1 for i in range(abs(s))])
            #activity.extend(['motionless' for i in range(abs(s))])

    seconds_index = [0]
    for s in seconds:
        seconds_index.append(seconds_index[-1] + s)
    del seconds_index[-1]
    #print(zip(seconds, seconds_index))
    #print(len(seconds), len(seconds_index))


    #print(min(seconds), min([1 for i in seconds]), min(range(len(seconds))), min(seconds), min(dividing_lines))

    #print(len(seconds), len(range(len(seconds))), len([1 for i in seconds]))

    #s2 = [float(i) / sum(seconds) * 100 for i in seconds]
    #print(s2[0])
    #s2[0] = 1000
    print(seconds_index)
    barlist = plt.bar(seconds_index, [1 for i in seconds], align='edge', color=activity) #width=seconds,
    '''
    for i in range(len(activity)):
        if activity[i] == 'absent':
            barlist[i].set_linewidth(seconds[i])
            barlist[i].set_color('royalblue')
        elif activity[i] == 'moving':
            barlist[i].set_linewidth(seconds[i])
            barlist[i].set_color('k')
        else:
            barlist[i].set_linewidth(seconds[i])
            barlist[i].set_color('indianred')
    '''
    #if dividing_lines is not None:
        #for xc in dividing_lines:
            #plt.axvline(x=xc, linewidth=4, color='white')


    #plt.bar(range(len(list_values)), list_values)
    #plt.xticks(range(len(list_values_names)), list_values_names, rotation=90)
    #plt.ylim(ymin, ymax)
    plt.xlabel(xtitle)
    plt.ylabel(ytitle)
    plt.title(title)
    plt.savefig(file_name, dpi=100)
    plt.clf()
    plt.close()

In [ ]:
import sys
import os

from Processor.PathAnalysis.pathmetrics import PathMetrics
from Processor.Utils.graphics import plot_path_bg, plot_histogram, plot_line
from Processor.Utils.fileutils import create_dir_check_exists
from Processor.Utils import constants

def main():
    experiment_directory = sys.argv[1]

    plots_dir = create_dir_check_exists(experiment_directory, 'plots')
    path_bg_dir = create_dir_check_exists(plots_dir, 'path_bg')
    distances_per_second_window_dir = create_dir_check_exists(plots_dir, 'distances_per_second_window')
    seconds_spent_in_perimeter_dir = create_dir_check_exists(plots_dir, 'seconds_spent_in_perimeter')
    consecutive_seconds_motionless_dir = create_dir_check_exists(plots_dir, 'consecutive_seconds_motionless')
    consecutive_seconds_motionless_path_dir = create_dir_check_exists(plots_dir, 'consecutive_seconds_motionless_path')

    heatmaps_dir = create_dir_check_exists(plots_dir, 'heatmaps')
    pm = PathMetrics(experiment_directory)
    pm.group_bee_data_into_nights_days()
    pm.generate_night_day_bgs()

    bee = pm.tag_class_night_day_metrics[constants.QUEEN_CLASS]
    for night_day in bee.keys():
        print(night_day)
        night_or_day_count = 0
        night_or_day_id = {'night': '_0_', 'day': '_1_'}
        for i in range(len(bee[night_day])):
            print(i)
            metrics = bee[night_day][i]
            bg_image = pm.night_day_bg_images[night_day][i]

            time_period_str = str(night_or_day_count) + night_or_day_id[night_day] + night_day

            file_name = os.path.join(path_bg_dir, time_period_str + '.png')
            plot_path_bg(metrics.x_paths, metrics.y_paths, bg_image, file_name)

            file_name = os.path.join(distances_per_second_window_dir, time_period_str + '.png')
            plot_histogram(metrics.all_distances_per_second_window, 'Distance per second', file_name)

            #file_name = os.path.join(seconds_spent_in_perimeter_dir, time_period_str + '.png')
            #plot_histogram(metrics['seconds_spent_in_perimeter'], 'Seconds spent in perimeter', file_name)

            seconds_motionless = []
            for motionless_data in metrics.all_motionless_data:
                seconds_motionless.append(motionless_data['seconds_motionless'])

            file_name = os.path.join(consecutive_seconds_motionless_dir, time_period_str + '.png')
            plot_histogram(seconds_motionless, 'Consecutive seconds motionless', file_name)

            file_name = os.path.join(consecutive_seconds_motionless_path_dir, time_period_str + '.png')
            plot_line(range(len(seconds_motionless)), seconds_motionless, 'Consecutive seconds motionless', 'Seconds in paths', 'Seconds motionless', 0, constants.SECONDS_IN_HOUR, file_name)

            for speed_group in metrics.cells_visited_speed_groups.keys():
                speed_group_heatmap_dir = create_dir_check_exists(heatmaps_dir, speed_group)
                file_name = os.path.join(speed_group_heatmap_dir, time_period_str)
                all_coords_spread, class_presence_spread = pm.calc_heatmaps(metrics.cells_visited_speed_groups[speed_group], bg_image, file_name)


            night_or_day_count += 1

if __name__ == "__main__":
    main()